# Transfer Learning Using an Encoder–Decoder 

### 🧠 Overview
We'll follow these steps:

1. **Train an Autoencoder** on the MNIST dataset to learn compressed representations of the images.
2. **Transfer the Encoder** part of the autoencoder to a new model.
3. **Train a Classifier** using the transferred encoder to classify MNIST digits.

This approach showcases how an encoder trained for one task (image reconstruction) can be repurposed for another (classification).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


### Define the Autoencoder

In [2]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


### Load the MNIST Dataset

In [3]:
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [01:51<00:00, 89.0kB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 109kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:04<00:00, 379kB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



### Train the Autoencoder

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

for epoch in range(5):
    for data, _ in train_loader:
        data = data.to(device)
        output = autoencoder(data)
        loss = criterion(output, data.view(-1, 28*28))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/5], Loss: {loss.item():.4f}')


Epoch [1/5], Loss: 0.0184
Epoch [2/5], Loss: 0.0123
Epoch [3/5], Loss: 0.0111
Epoch [4/5], Loss: 0.0105
Epoch [5/5], Loss: 0.0088


###  Define the Classifier Using the Pretrained Encoder

In [5]:
class Classifier(nn.Module):
    def __init__(self, encoder):
        super(Classifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(64, 10)
        )

    def forward(self, x):
        with torch.no_grad():
            encoded = self.encoder(x)
        output = self.classifier(encoded)
        return output


### Prepare the Classifier

In [6]:
# Freeze encoder parameters
for param in autoencoder.encoder.parameters():
    param.requires_grad = False

classifier = Classifier(autoencoder.encoder).to(device)
criterion_cls = nn.CrossEntropyLoss()
optimizer_cls = torch.optim.Adam(classifier.classifier.parameters(), lr=1e-3)


### Train the Classifier

In [7]:
for epoch in range(5):
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = classifier(data)
        loss = criterion_cls(outputs, labels)
        optimizer_cls.zero_grad()
        loss.backward()
        optimizer_cls.step()
    print(f'Epoch [{epoch+1}/5], Loss: {loss.item():.4f}')


Epoch [1/5], Loss: 0.8399
Epoch [2/5], Loss: 0.4020
Epoch [3/5], Loss: 0.3212
Epoch [4/5], Loss: 0.5398
Epoch [5/5], Loss: 0.2340
